# Solving simple artificial neural network to optimality in EAGO 

[Matthew Wilhelm](https://psor.uconn.edu/person/matthew-wilhelm/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

In [1,2], a surrogate ANN model of bioreactor productivity was constructed by fitting results from computationally expensive CFD simulations. The author then optimized this surrogate model to obtain ideal processing conditions. This optimization problem is given by:

$
\begin{align}
    \label{prob:ANN1}
    \max_{\mathbf x\in X} B_2 + \sum_{i=1}^{3}\frac{2D_{i}}{1+\exp(-2y_{i})} \qquad \text{where} \qquad y_{i} =  B_i + \sum_{i=1}^{3}\sum_{j=1}^{8} W_{ij}x_{i}\\
\end{align}
$

We'll repeat this exercise using JuMP and EAGO.

### Input parameters

In the first block, we input parameters values supplied in the paper for $W$, $B$, $D$, and $X$ into Julia as simple array objects.

In [1]:
# Box constraints for input variables
xLBD = [0.623   0.093   0.259   6.56   1114   0.013   0.127   0.004]
xUBD = [5.89    0.5     1.0     90     25000  0.149   0.889   0.049]

# Weights associated with the hidden layer
W = [ 0.54  -1.97  0.09  -2.14  1.01  -0.58  0.45  0.26;
     -0.81  -0.74  0.63  -1.60 -0.56  -1.05  1.23  0.93;
     -0.11  -0.38 -1.19   0.43  1.21   2.78 -0.06  0.40]

# Weights associated with the output layer
D = [-0.91 0.11 0.52]

# Bias associated with the hidden layer
B1 = [-2.698 0.012 2.926]

# Bias associated with the output layer
B2 = -0.46

-0.46

### Construct the JuMP model and optimize

We now formulate the problem using standard JuMP[3] syntax and optimize it. Note that we are forming an NLexpression object to handle the summation term to keep the code visually simple but this could be placed directly in the JuMP expressions instead.

In [2]:
using JuMP, EAGO

# Model construction
model = Model(with_optimizer(EAGO.Optimizer, absolute_tolerance = 0.001))
@variable(model, xLBD[i] <= x[i=1:8] <= xUBD[i])
@NLexpression(model, prop[i=1:3], B1[i] + sum(W[i,j]*x[i] for j in 1:8))
@NLobjective(model, Max, B2 + sum(D[i]*(2/(1+exp(-2*prop[i]))) for i=1:3))

# Solves the model
optimize!(model)

┌ Info: Recompiling stale cache file C:\Users\wilhe\.julia\compiled\v1.0\EAGO\t0N0c.ji for EAGO [bb8be931-2a91-5aca-9f87-79e1cb69959a]
└ @ Base loading.jl:1190
┌ Warning: Package EAGO does not have CPLEX in its dependencies:
│ - If you have EAGO checked out for development and have
│   added CPLEX as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with EAGO
└ Loading CPLEX into EAGO from project dependency, future warnings for EAGO are suppressed.


start set to default
end set to default
during load
Node ID: 1, Lower Bound: -Inf, Lower Variable Bounds:
             [0.623, 0.093, 0.259, 6.56, 1114.0, 0.013, 0.127, 0.004], Upper Variable Bounds: [5.89, 0.5, 1.0, 90.0, 25000.0, 0.149, 0.889, 0.049]
started initial preprocessing
finished initial preprocessing
during load
post load
started evaluating subexpressions
k: 27, node type: Variable[3]
k: 26, node type: Constant = 0.4
k: 25, node type: +( node[25], node[26], )
k: 24, node type: Variable[3]
k: 23, node type: Constant = -0.06
k: 22, node type: +( node[23], node[24], )
k: 21, node type: Variable[3]
k: 20, node type: Constant = 2.78
k: 19, node type: +( node[21], node[22], )
k: 18, node type: Variable[3]
k: 17, node type: Constant = 1.21
k: 16, node type: +( node[19], node[20], )
k: 15, node type: Variable[3]
k: 14, node type: Constant = 0.43
k: 13, node type: +( node[17], node[18], )
k: 12, node type: Variable[3]
k: 11, node type: Constant = -1.19
k: 10, node type: +( node[15],

KeyError: KeyError: key 12 not found

### Retrieve results

We then recover the objective value, the solution value, and termination status codes using standard JuMP syntax.

In [3]:
# Access calculated values
fval = JuMP.objective_value(model)
xsol = JuMP.value.(x)
status_term = JuMP.termination_status(model)
status_prim = JuMP.primal_status(model)

println("EAGO terminated with a status of $status_term and a result code of $status_prim")
println("The optimal value is: $fval, the solution found is $xsol.")

JuMP.OptimizeNotCalled: JuMP.OptimizeNotCalled()

### Reference:
1. J. D. Smith, A. A. Neto, S. Cremaschi, and D. W. Crunkleton, CFD-based optimization of a flooded bed algae bioreactor, *Industrial & Engineering Chemistry Research*, 52 (2012), pp. 7181–7188
2. A. M. Schweidtmann and A. Mitsos. Global Deterministic Optimization with Artificial Neural Networks Embedded [https://arxiv.org/pdf/1801.07114.pdf](https://arxiv.org/pdf/1801.07114.pdf)
3. Iain Dunning and Joey Huchette and Miles Lubin. JuMP: A Modeling Language for Mathematical Optimization, *SIAM Review*, 59 (2017), pp. 295-320.